# EPL Fantasy 6-Week Predictor Data Prep
The format of EPL data has changed across the years, so I go through and pull weekly data for each year and combine for previous and next 6-weeks. 

Player and team performance can be dip and rise throughout the season. So instead of predicting players for the entire season, I narrow my focus down to the next 6-weeks in bring in other variables like average difficulty of teams they'll play.

This whole cleaning process could be looped but there are some differences in data structure across years I decided to address individually.

## Table of Contents

* [Pull Data](#pulldata)
    * [2016-2017 Season](#2016_2017_season)
    * [2017-2018 Season](#2017_2018_season)
    * [2018-2019 Season](#2018_2019_season)
    * [2019-2020 Season](#2019_2020_season)
* [Combine Data](#combinedata)

In [3]:
import pandas as pd
import numpy as np
import requests

# Pull Data <a class="anchor" id="pulldata"></a>
## 2016-2017 Season <a class="anchor" id="2016_2017_season"></a>
/vaastav/ has been scraping data from EPL website for years now, so I loop through his scraped csv's. Here we set our target to the previous 6 weeks and next 6 weeks

In [4]:
##First 4 GW of 16-17
url = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw15.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw16.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw17.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw18.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw19.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw20.csv"]

##Next 6 GW of 16-17
url2 = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw21.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw22.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw23.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw24.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw25.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw26.csv"]

In [5]:
url3 = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/players_raw.csv"
players = pd.read_csv(url3, encoding = "ISO-8859-1", error_bad_lines=False)
players = pd.DataFrame(players)
players = players[['element_type','id']]


url4 = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/player_idlist.csv"
id_list = pd.read_csv(url4, encoding = 'utf-8', error_bad_lines=False)
id_list = pd.DataFrame(id_list)
# id_list.head()

print(id_list[id_list["id"] == 482])

    first_name second_name   id
481  Abdoulaye    Doucouré  482


I calculated fixture difficulty for each of the years. We also keep only the dimensions that appear across all years of data. And group them into a 6-week total or average.

In [6]:
##EPL Fixture Difficulty is in the github project repo
fix_diff = pd.read_csv("https://raw.githubusercontent.com/tprice90/epl_fantasy_predictor/main/Data%20Prep/epl_difficulty_16_17.csv")
fix_diff = pd.DataFrame(fix_diff)

df_16_17 = pd.DataFrame()
for data in url:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_16_17 = df_16_17.append(df)
    
df_n6 = pd.DataFrame()
for data in url2:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_n6 = df_n6.append(df)

df_n6 = df_n6.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')
df_n6 = df_n6.groupby('name', as_index=False).agg({'total_points':'sum',
                                                  'opponent_difficulty':'mean'})
df_n6.rename(columns={'total_points':'n6_total_points','opponent_difficulty':'n6_opponent_difficulty'}, inplace=True)
df_16_17 = df_16_17.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')

df_16_17_agg = df_16_17.groupby(['name'], as_index=False).agg({'assists':'sum',
                                                             'bonus':'sum',
                                                             'bps':'sum',
                                                             'clean_sheets':'sum',                                                             
                                                              'goals_conceded':'sum',
                                                              'goals_scored':'sum',
                                                              'minutes':'sum',
                                                              'own_goals':'sum',
                                                              'penalties_missed':'sum',
                                                              'penalties_saved':'sum',
                                                              'red_cards':'sum',
                                                              'saves':'sum',
                                                              'selected':'sum',
                                                              'total_points':'sum',
                                                              'transfers_balance':'sum',
                                                              'was_home':'sum',
                                                              'yellow_cards':'sum',
                                                              'opponent_difficulty':'mean',
                                                              'round':'count'})
df_16_17_agg['round'] = max(df_16_17_agg['round'])
##Get Last Value for players in GW range
df_16_17_last = df_16_17.groupby('name',as_index=False).last()
df_16_17_last = df_16_17_last[['name','element','creativity','ict_index','influence','threat','value']]

df_16_17_first = df_16_17.groupby('name',as_index=False).first()
# df_16_17_first = df_16_17_first[['name','id']]

df_16_17 = df_16_17_agg.merge(df_16_17_last, left_on='name', right_on='name')
df_16_17 = df_16_17.merge(df_n6, left_on='name', right_on='name')
df_16_17['perc_min_played'] = df_16_17['minutes']/(df_16_17['round']*90)
df_16_17 = df_16_17.drop(columns=['minutes','round'], axis=0)

There are a few dimensions I need to pull from the players raw data but need to standardize the format of names in both data sources

In [7]:
new = df_16_17["name"].str.split("_", expand = True) 

df_16_17["first_name"]= new[0] 
  
# making separate last name column from new data frame 
df_16_17["second_name"]= new[1] 

df_16_17.head()

df_16_17 = pd.merge(df_16_17, id_list,  how='left', left_on=['first_name','second_name'], right_on=['first_name','second_name'])
df_16_17 = df_16_17.drop(columns=['first_name','second_name'], axis=0)
df_16_17 = df_16_17.merge(players, left_on='id', right_on='id')

## 2017-2018 Season <a class="anchor" id="2017_2018_season"></a>


In [8]:
##First 4 GW of 17-18
url = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw15.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw16.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw17.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw18.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw19.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw20.csv"]

##Next 6 GW of 17-18
url2 = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw21.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw22.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw23.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw24.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw25.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw26.csv"]

In [9]:
url3 = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/players_raw.csv"
players = pd.read_csv(url3, encoding = "ISO-8859-1", error_bad_lines=False)
players = pd.DataFrame(players)
players = players[['element_type','id']]

url4 = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/player_idlist.csv"
id_list = pd.read_csv(url4, encoding = 'utf-8', error_bad_lines=False)
id_list = pd.DataFrame(id_list)
# id_list.head()

print(id_list[id_list["id"] == 482])

    first_name second_name   id
180      Jairo   Riedewald  482


In [10]:
fix_diff = pd.read_csv("https://raw.githubusercontent.com/tprice90/epl_fantasy_predictor/main/Data%20Prep/epl_difficulty_17_18.csv")
fix_diff = pd.DataFrame(fix_diff)

df_17_18 = pd.DataFrame()
for data in url:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_17_18 = df_17_18.append(df)
    
df_n6 = pd.DataFrame()
for data in url2:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_n6 = df_n6.append(df)

df_n6 = df_n6.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')
df_n6 = df_n6.groupby('name', as_index=False).agg({'total_points':'sum',
                                                  'opponent_difficulty':'mean'})
df_n6.rename(columns={'total_points':'n6_total_points','opponent_difficulty':'n6_opponent_difficulty'}, inplace=True)
df_17_18 = df_17_18.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')

df_17_18_agg = df_17_18.groupby(['name'], as_index=False).agg({'assists':'sum',
                                                             'bonus':'sum',
                                                             'bps':'sum',
                                                             'clean_sheets':'sum',                                                             
                                                              'goals_conceded':'sum',
                                                              'goals_scored':'sum',
                                                              'minutes':'sum',
                                                              'own_goals':'sum',
                                                              'penalties_missed':'sum',
                                                              'penalties_saved':'sum',
                                                              'red_cards':'sum',
                                                              'saves':'sum',
                                                              'selected':'sum',
                                                              'total_points':'sum',
                                                              'transfers_balance':'sum',
                                                              'was_home':'sum',
                                                              'yellow_cards':'sum',
                                                              'opponent_difficulty':'mean',
                                                              'round':'count'})
df_17_18_agg['round'] = max(df_17_18_agg['round'])
##Get Last Value for players in GW range
df_17_18_last = df_17_18.groupby('name',as_index=False).last()
df_17_18_last = df_17_18_last[['name','element','creativity','ict_index','influence','threat','value']]

df_17_18_first = df_17_18.groupby('name',as_index=False).first()
# df_17_18_first = df_17_18_first[['name','id']]

df_17_18 = df_17_18_agg.merge(df_17_18_last, left_on='name', right_on='name')
df_17_18 = df_17_18.merge(df_n6, left_on='name', right_on='name')
df_17_18['perc_min_played'] = df_17_18['minutes']/(df_17_18['round']*90)
df_17_18 = df_17_18.drop(columns=['minutes','round'], axis=0)

In [11]:
new = df_17_18["name"].str.split("_", expand = True) 

df_17_18["first_name"]= new[0] 
  
# making separate last name column from new data frame 
df_17_18["second_name"]= new[1] 

df_17_18.head()

df_17_18 = pd.merge(df_17_18, id_list,  how='left', left_on=['first_name','second_name'], right_on=['first_name','second_name'])
df_17_18 = df_17_18.drop(columns=['first_name','second_name'], axis=0)
df_17_18 = df_17_18.merge(players, left_on='id', right_on='id')

## 2018-2019 Season <a class="anchor" id="2018_2019_season"></a>

In [12]:
##First 4 GW of 2018-19
url = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw15.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw16.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw17.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw18.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw19.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw20.csv"]

##Next 6 GW of 18-19
url2 = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw21.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw22.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw23.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw24.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw25.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw26.csv"]


In [13]:
url3 = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/players_raw.csv"
players = pd.read_csv(url3, encoding = "ISO-8859-1", error_bad_lines=False)
players = pd.DataFrame(players)
players = players[['element_type','id']]

url4 = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/player_idlist.csv"
id_list = pd.read_csv(url4, encoding = 'utf-8', error_bad_lines=False)
id_list = pd.DataFrame(id_list)
# id_list.head()

print(id_list[id_list["id"] == 482])

    first_name second_name   id
451     Fabian       Schär  482


In [14]:
fix_diff = pd.read_csv("https://raw.githubusercontent.com/tprice90/epl_fantasy_predictor/main/Data%20Prep/epl_difficulty_18_19.csv")
fix_diff = pd.DataFrame(fix_diff)


df_18_19 = pd.DataFrame()
for data in url:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_18_19 = df_18_19.append(df)
    
df_n6 = pd.DataFrame()
for data in url2:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_n6 = df_n6.append(df)

df_n6 = df_n6.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')
df_n6 = df_n6.groupby('name', as_index=False).agg({'total_points':'sum',
                                                  'opponent_difficulty':'mean'})
df_n6.rename(columns={'total_points':'n6_total_points','opponent_difficulty':'n6_opponent_difficulty'}, inplace=True)
df_18_19 = df_18_19.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')

df_18_19_agg = df_18_19.groupby(['name'], as_index=False).agg({'assists':'sum',
                                                             'bonus':'sum',
                                                             'bps':'sum',
                                                             'clean_sheets':'sum',                                                             
                                                              'goals_conceded':'sum',
                                                              'goals_scored':'sum',
                                                              'minutes':'sum',
                                                              'own_goals':'sum',
                                                              'penalties_missed':'sum',
                                                              'penalties_saved':'sum',
                                                              'red_cards':'sum',
                                                              'saves':'sum',
                                                              'selected':'sum',
                                                              'total_points':'sum',
                                                              'transfers_balance':'sum',
                                                              'was_home':'sum',
                                                              'yellow_cards':'sum',
                                                              'opponent_difficulty':'mean',
                                                              'round':'count'})
df_18_19_agg['round'] = max(df_18_19_agg['round'])
##Get Last Value for players in GW range
df_18_19_last = df_18_19.groupby('name',as_index=False).last()
df_18_19_last = df_18_19_last[['name','element','creativity','ict_index','influence','threat','value']]
	
df_18_19_first = df_18_19.groupby('name',as_index=False).first()
# df_18_19_first = df_18_19_first[['name','id']]

df_18_19 = df_18_19_agg.merge(df_18_19_last, left_on='name', right_on='name')
df_18_19 = df_18_19.merge(df_n6, left_on='name', right_on='name')
df_18_19['perc_min_played'] = df_18_19['minutes']/(df_18_19['round']*90)
df_18_19 = df_18_19.drop(columns=['minutes','round'], axis=0)

In [15]:
new = df_18_19["name"].str.split("_", expand = True) 

df_18_19["first_name"]= new[0] 
  
# making separate last name column from new data frame 
df_18_19["second_name"]= new[1] 

df_18_19.head()

df_18_19 = pd.merge(df_18_19, id_list,  how='left', left_on=['first_name','second_name'], right_on=['first_name','second_name'])
df_18_19 = df_18_19.drop(columns=['first_name','second_name'], axis=0)
df_18_19 = df_18_19.merge(players, left_on='id', right_on='id')

## 2019-2020 Season <a class="anchor" id="2019_2020_season"></a>

In [16]:
##First 4 GW of 2019-20
url = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw15.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw16.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw17.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw18.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw19.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw20.csv"]

##Next 6 GW of 19-20
url2 = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw21.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw22.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw23.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw24.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw25.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw26.csv"]


In [17]:
url3 = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/players_raw.csv"
players = pd.read_csv(url3, encoding = "ISO-8859-1", error_bad_lines=False)
players = pd.DataFrame(players)
players = players[['element_type','id']]


url4 = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/player_idlist.csv"
id_list = pd.read_csv(url4, encoding = "ISO-8859-1", error_bad_lines=False)
id_list = pd.DataFrame(id_list)

In [18]:
fix_diff = pd.read_csv("https://raw.githubusercontent.com/tprice90/epl_fantasy_predictor/main/Data%20Prep/epl_difficulty_19_20.csv")
fix_diff = pd.DataFrame(fix_diff)

df_19_20 = pd.DataFrame()
for data in url:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_19_20 = df_19_20.append(df)
    
df_n6 = pd.DataFrame()
for data in url2:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_n6 = df_n6.append(df)

df_n6 = df_n6.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')
df_n6 = df_n6.groupby('name', as_index=False).agg({'total_points':'sum',
                                                  'opponent_difficulty':'mean'})
df_n6.rename(columns={'total_points':'n6_total_points','opponent_difficulty':'n6_opponent_difficulty'}, inplace=True)
df_19_20 = df_19_20.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')

df_19_20_agg = df_19_20.groupby(['name'], as_index=False).agg({'assists':'sum',
                                                             'bonus':'sum',
                                                             'bps':'sum',
                                                             'clean_sheets':'sum',                                                             
                                                              'goals_conceded':'sum',
                                                              'goals_scored':'sum',
                                                              'minutes':'sum',
                                                              'own_goals':'sum',
                                                              'penalties_missed':'sum',
                                                              'penalties_saved':'sum',
                                                              'red_cards':'sum',
                                                              'saves':'sum',
                                                              'selected':'sum',
                                                              'total_points':'sum',
                                                              'transfers_balance':'sum',
                                                              'was_home':'sum',
                                                              'yellow_cards':'sum',
                                                              'opponent_difficulty':'mean',
                                                              'round':'count'})

df_19_20_agg['round'] = max(df_19_20_agg['round'])
##Get Last Value for players in GW range
df_19_20_last = df_19_20.groupby('name',as_index=False).last()
df_19_20_last = df_19_20_last[['name','element','creativity','ict_index','influence','threat','value']]
	
df_19_20_first = df_19_20.groupby('name',as_index=False).first()
# df_19_20_first = df_19_20_first[['name','id']]

df_19_20 = df_19_20_agg.merge(df_19_20_last, left_on='name', right_on='name')
df_19_20 = df_19_20.merge(df_n6, left_on='name', right_on='name')
df_19_20['perc_min_played'] = df_19_20['minutes']/(df_19_20['round']*90)
df_19_20 = df_19_20.drop(columns=['minutes','round'], axis=0)

In [19]:
new = df_19_20["name"].str.split("_", expand = True) 

df_19_20["first_name"]= new[0] 
  
# making separate last name column from new data frame 
df_19_20["second_name"]= new[1] 

df_19_20.head()

df_19_20 = pd.merge(df_19_20, id_list,  how='left', left_on=['first_name','second_name'], right_on=['first_name','second_name'])
df_19_20 = df_19_20.drop(columns=['first_name','second_name'], axis=0)
df_19_20 = df_19_20.merge(players, left_on='id', right_on='id')

# Combine Data <a class="anchor" id="combinedata"></a>

In [20]:
df = df_16_17.append([df_17_18,df_18_19,df_19_20])
#df['element'] = df['element'].apply(str)
df = pd.get_dummies(df,columns=['element_type'])
df_to_export = df
df = df.drop(columns=['name','element','id'], axis=0)
df.dtypes

assists                     int64
bonus                       int64
bps                         int64
clean_sheets                int64
goals_conceded              int64
goals_scored                int64
own_goals                   int64
penalties_missed            int64
penalties_saved             int64
red_cards                   int64
saves                       int64
selected                    int64
total_points                int64
transfers_balance           int64
was_home                    int64
yellow_cards                int64
opponent_difficulty       float64
creativity                float64
ict_index                 float64
influence                 float64
threat                    float64
value                       int64
n6_total_points             int64
n6_opponent_difficulty    float64
perc_min_played           float64
element_type_1              uint8
element_type_2              uint8
element_type_3              uint8
element_type_4              uint8
dtype: object

In [21]:
df.head()

,assists,bonus,bps,clean_sheets,goals_conceded,goals_scored,own_goals,penalties_missed,penalties_saved,red_cards,...,influence,threat,value,n6_total_points,n6_opponent_difficulty,perc_min_played,element_type_1,element_type_2,element_type_3,element_type_4
0,0,1,102,2,6,0,0,0,0,0,...,7.2,8.0,53,12,2.500000,1.000000,0,1,0,0
1,1,0,62,2,2,0,0,0,0,0,...,0.0,18.0,55,1,2.166667,0.577778,0,0,1,0
2,0,0,26,0,3,0,0,0,0,0,...,7.2,37.0,76,6,2.400000,0.233333,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,45,0,2.166667,0.000000,0,0,1,0
4,0,0,18,0,6,0,0,0,0,0,...,3.6,18.0,46,7,2.833333,0.333333,0,0,1,0
